In [1]:
import os
import numpy as np
import json
import neuroglancer
from matplotlib import pyplot as plt

viewer = neuroglancer.Viewer()
viewer

http://127.0.0.1:45041/v/465af6b3950b69cd08d2e2be11ea84db1af64bd2/

In [2]:
color_filepath = os.path.join('./', 'neuroglancer/contours/json_cache', 'struct_to_color.json')
with open(color_filepath, 'r') as json_file:
    colors = json.load(json_file)
print('Color maps:', colors)

Color maps: {'10N_L': 1, '10N_R': 1, '12N': 2, '3N_L': 3, '3N_R': 3, '4N_L': 4, '4N_R': 4, '5N_L': 5, '5N_R': 5, '6N_L': 6, '6N_R': 6, '7N_L': 7, '7N_R': 7, 'AMB_L': 8, 'AMB_R': 8, 'AP': 9, 'DC_L': 10, 'DC_R': 10, 'IC': 11, 'LC_L': 12, 'LC_R': 12, 'LRT_L': 13, 'LRT_R': 13, 'PBG_L': 14, 'PBG_R': 14, 'PN_L': 15, 'PN_R': 15, 'RMC_L': 16, 'RMC_R': 16, 'RTTG': 17, 'SC': 18, 'SNC_L': 19, 'SNC_R': 19, 'SNR_L': 20, 'SNR_R': 20, 'SP5C_L': 21, 'SP5C_R': 21, 'SP5I_L': 22, 'SP5I_R': 22, 'SP5O_L': 23, 'SP5O_R': 23, 'TZ_L': 24, 'TZ_R': 24, 'VCA_L': 25, 'VCA_R': 25, 'VCP_L': 26, 'VCP_R': 26, 'VLL_L': 27, 'VLL_R': 27}


In [3]:
VOL_DIR = '/net/birdstore/Active_Atlas_Data/copied_from_S3/mousebrainatlas-data/CSHL_volumes/atlasV7/atlasV7_10.0um_scoreVolume/score_volumes'
files = os.listdir(VOL_DIR)
numpy_files = [f for f in files if f.endswith('.npy') and 'surround' not in f]
print('Number of numpy files:', len(numpy_files))

Number of numpy files: 51


## MD589 reference

In [4]:
MD589_VOLUME_PATH = '/net/birdstore/Active_Atlas_Data/data_root/CSHL_volumes/MD589/MD589_full_filled.npy'
MD589 = np.load(MD589_VOLUME_PATH)
MD589 = np.swapaxes(MD589, 0, 2)

np.unique(MD589, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28], dtype=uint8),
 array([422952046,     50371,    339451,    166093,   8322571,   5684631,
           435298,      3478,    795811,    379305,     26766,    149034,
           564906,   1266877,    240473,    633955,    336884,    143404,
          2108801,     98317,      6743,   1157904,   1949655,   2419452,
            44056,    114926,    763465,    414515,    726460]))

In [15]:
layer = neuroglancer.SegmentationLayer(
    source=neuroglancer.LocalVolume(
        data=MD589, 
        dimensions=neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='um', scales=[5, 5, 20]), 
        voxel_offset=(0, 0, 0)
    ),
)

with viewer.txn() as s:
#     s.layers.clear()
    s.layers['layer'] = layer

## Aggregation

In [30]:
x_length = 2000
y_length = 2000
z_length = 500
full_brain_volume_annotated = np.zeros((z_length, y_length, x_length), dtype=np.uint8)

for n in numpy_files:
    prefix = os.path.splitext(n)[0]
    structure = prefix.split('_', 3)[3]
    
    if structure not in ['SC']:
        continue
    
    try:
        color = colors[structure.upper()]
    except:
        sided = '{}_R'.format(structure.upper())
        color = colors[sided]

    volume_filename = os.path.join(VOL_DIR, n)
    volume_input = np.load(volume_filename)
    volume_input = np.swapaxes(volume_input, 0, 2)
    volume_input = np.swapaxes(volume_input, 1, 2)

    volume_nonzero_indices = volume_input > 0.9
    volume_input[volume_nonzero_indices] = color
    structure_volume = volume_input.astype(np.uint8)

    origin_filename = os.path.join(VOL_DIR, '{}_origin_wrt_canonicalAtlasSpace.txt'.format(prefix))
    origin_wrt = np.loadtxt(origin_filename)
    x,y,z = origin_wrt
    x_start = int(x) + x_length // 2
    y_start = int(y) + y_length // 2
    z_start = 0
    x_end = x_start + structure_volume.shape[2]
    y_end = y_start + structure_volume.shape[1]
    z_end = z_start + structure_volume.shape[0]

    try:
        full_brain_volume_annotated[z_start:z_end, y_start:y_end,x_start:x_end] = structure_volume
        print('Fit\t',str(structure).ljust(8), str(color).rjust(2), end="\t")
        print('shape', str(structure_volume.shape).rjust(18), end=", ")
        print('x range', str(x_start).rjust(4), str(x_end).rjust(4), end=", ")
        print('y range', str(y_start).rjust(4), str(y_end).rjust(4), end=", ")
        print('z range', str(z_start).rjust(4), str(z_end).rjust(4))
    except:
        print('Error\t',str(structure).ljust(8), str(color).rjust(2), end="\t")
        print('shape', str(structure_volume.shape).rjust(18), end=", ")
        print('x range', str(x_start).rjust(4), str(x_end).rjust(4), end=", ")
        print('y range', str(y_start).rjust(4), str(y_end).rjust(4), end=", ")
        print('z range', str(z_start).rjust(4), str(z_end).rjust(4))
        
full_brain_volume_annotated = np.swapaxes(full_brain_volume_annotated, 0, 2)

Fit	 SC       18	shape    (377, 176, 238), x range  752  990, y range  683  859, z range    0  377


In [31]:
test_layer = neuroglancer.SegmentationLayer(
    source = neuroglancer.LocalVolume(
        data=full_brain_volume_annotated, 
        dimensions=neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='um', scales=[10, 10, 10]), 
        voxel_offset=(0, 0, 0)
    ),
)

with viewer.txn() as s:
    s.layers.clear()
    s.layers['aggregate'] = test_layer

## Individual volume 

In [8]:
for file in numpy_files:
    volume_filename = os.path.join(VOL_DIR, file)
    volume_input = np.load(volume_filename)
    volume_input[volume_input > 0.9] = 1
    volume_input = np.swapaxes(volume_input, 0, 2)
    
    dimensions = neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='um', scales=[10, 10, 10])
    
    prefix = os.path.splitext(file)[0]
    origin_filename = os.path.join(VOL_DIR, '{}_origin_wrt_canonicalAtlasSpace.txt'.format(prefix))
    origin_wrt = np.loadtxt(origin_filename)
    
    volume_layer = neuroglancer.SegmentationLayer(
        source = neuroglancer.LocalVolume(data=volume_input, dimensions=dimensions, voxel_offset=[0, 0, 0]),
        shader = """
            void main() {
                float v = toNormalized(getDataValue(0)) * 255.0;
                emitRGBA(vec4(v, 0.0, 0.0, v));
            }
        """
    )
    with viewer.txn() as s:
        s.layers[prefix.split('_', 3)[3]] = volume_layer